# Matthew Przybyla
# NLP Homework 8

In [809]:
# print out the versions of libraries, python version, and environment

from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os
import urllib
import bs4
from collections import Counter
from sklearn.cluster import KMeans
import sys; print("Python", sys.version)
import pandas as pd; print("pandas", pd.__version__)
import numpy as np; print("numpy", np.__version__)
import sklearn; print("sklearn", sklearn.__version__)
import requests; print("requests", requests.__version__)
import nltk; print("nltk", nltk.__version__)
import re; print('re', re.__version__)
from nltk.sentiment.vader import SentimentIntensityAnalyzer; print("SentimentIntensityAnalyzer", nltk.__version__)
from sklearn.metrics.pairwise import cosine_similarity; print("sklearn.metrics.pairwise", sklearn.__version__)
from sklearn.feature_extraction.text import TfidfVectorizer; print("sklearn.feature_extraction.text", sklearn.__version__)
from requests import get; print("requests", requests.__version__)
from urllib.parse import urljoin; print("urllib.parse", urllib.request.__version__)
from bs4 import BeautifulSoup; print("bs4", bs4.__version__)
import nltk.stem.snowball; print("nltk.stem.snowball", nltk.__version__)


Python 3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 14:01:38) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
pandas 0.20.3
numpy 1.13.3
sklearn 0.20.0
requests 2.14.2
nltk 3.3
re 2.2.1
SentimentIntensityAnalyzer 3.3
sklearn.metrics.pairwise 0.20.0
sklearn.feature_extraction.text 0.20.0
requests 2.14.2
urllib.parse 3.6
bs4 4.6.3
nltk.stem.snowball 3.3


# Review Data

In [810]:
# Perform a vocabulary-based sentiment analysis of the movie reviews you used in 
# homework 5 and homework 7, by doing the following:

In [855]:
hw7data = pd.read_csv('/Users/MatthewPrzybyla/Downloads/hw7reviewdata.csv',encoding="latin-1")

In [856]:
reviewdata = hw7data.drop(['Unnamed: 0','type','label', 'file'],axis=1)
reviewdata.columns = ["review"]
reviewdata.head()

,review
0,Once again Mr. Costner has dragged out a movie...
1,This is an example of why the majority of acti...
2,"First of all I hate those moronic rappers, who..."
3,Not even the Beatles could write songs everyon...
4,Brass pictures (movies is not a fitting word f...


In [857]:
train_dataRD = reviewdata[:1000]
test_dataRD = reviewdata[1000:]

In [858]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [859]:
stemmer = SnowballStemmer("english")

In [860]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in train_dataRD['review']:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'review', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [861]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 228823 items in vocab_frame


In [862]:
def tokens(x):
    return x.split(',')

In [863]:
tfidf_vect= TfidfVectorizer( tokenizer=tokens ,use_idf=True, smooth_idf=True, sublinear_tf=False)

In [864]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(train_dataRD['review']) #fit the vectorizer to synopses

In [865]:
terms = tfidf_vectorizer.get_feature_names()

In [866]:
dist = 1 - cosine_similarity(tfidf_matrix)

In [867]:
num_clusters = 4
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

CPU times: user 51.1 s, sys: 725 ms, total: 51.8 s
Wall time: 13.9 s


In [868]:
#cluster = {'clusters': clusters}

In [869]:
reviewData = pd.DataFrame(train_dataRD, index = clusters)

In [870]:
reviewData.head()

,review
3,Not even the Beatles could write songs everyon...
2,"First of all I hate those moronic rappers, who..."
0,Once again Mr. Costner has dragged out a movie...
1,This is an example of why the majority of acti...
2,"First of all I hate those moronic rappers, who..."


In [871]:
reviewData.loc[:,'cluster'] = km.labels_

In [872]:
reviewData.head()

,review,cluster
3,Not even the Beatles could write songs everyon...,3
2,"First of all I hate those moronic rappers, who...",2
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,1
2,"First of all I hate those moronic rappers, who...",2


In [873]:
one = pd.DataFrame(reviewData[reviewData['cluster'] == 0])
two = pd.DataFrame(reviewData[reviewData['cluster'] == 1])
three = pd.DataFrame(reviewData[reviewData['cluster'] == 2])
four = pd.DataFrame(reviewData[reviewData['cluster'] == 3])

In [874]:
reviewData['cluster'].unique()

array([3, 2, 0, 1])

In [875]:
#you can also do this if you want on 1 line
# get the total number of movies per cluster
c = Counter(clusters)
print(c.items())

dict_items([(3, 180), (2, 298), (0, 271), (1, 251)])


In [876]:
print('Cluster 1', one['cluster'].count())
print('Cluster 2', two['cluster'].count())
print('Cluster 3', three['cluster'].count())
print('Cluster 4', four['cluster'].count())

Cluster 1 271
Cluster 2 251
Cluster 3 298
Cluster 4 180


# Question 1

In [491]:
# 1. In Python, load one of the sentiment vocabularies referenced in the textbook, 
# and run the sentiment analyzer as explained in the corresponding reference. 
# Add words to the sentiment vocabulary, if you think you need to, to better 
# fit your particular text collection.

In [877]:
# read in the data (it takes a bit of time)
# download dataset from http://ai.stanford.edu/~amaas/data/sentiment/
# change the directory path to your path 
# path should point to the unzipped directory with reviews
# I had unzipped it to E:/aclImdb
labels = {'pos': 'positive', 'neg': 'negative'}

dataset = pd.DataFrame()
for directory in ('test', 'train'):
    for sentiment in ('pos', 'neg'):
        path ='/Users/MatthewPrzybyla/Downloads/aclImdb/{}/{}'.format(directory, sentiment)
        for review_file in os.listdir(path):
            with open(os.path.join(path, review_file), 'r') as input_file:
                review = input_file.read()
            dataset = dataset.append([[review, labels[sentiment]]], 
                                     ignore_index=True)

dataset.columns = ['review', 'sentiment']

indices = dataset.index.tolist()
np.random.shuffle(indices)
indices = np.array(indices)

dataset = dataset.reindex(index=indices)

dataset.to_csv('movie_reviews.csv', index=False)

In [878]:
dataset.head()

,review,sentiment
25523,I enjoyed this film yet hated it because I wan...,positive
31904,"Of all the football films I have watched, this...",positive
5056,I just saw this wonderfully filmed movie that ...,positive
16045,"Anyone remember the first CKY, CKY2K etc..? Ba...",negative
10596,I loved this movie when I was a kid. I saw it ...,positive


In [879]:
# split up dataset
train_data = dataset[:35000]
test_data = dataset[35000:]

test_reviews = np.array(test_data['review'])
test_sentiments = np.array(test_data['sentiment'])


sample_docs = [100, 5817, 7626, 7356, 1008, 7155, 3533, 13010]
sample_data = [(test_reviews[index],
                test_sentiments[index])
                  for index in sample_docs]

In [880]:
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [881]:
def analyze_sentiment_vader_lexicon(review, 
                                    threshold=0.1,
                                    verbose=False):
    # pre-process text
    review = clean_text(review)
    #review = html_parser.unescape(review)
    #review = strip_html(review)
    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 'positive' if agg_score >= threshold\
                                   else 'negative'
    if verbose:
        # display detailed sentiment statistics
        positive = str(round(scores['pos'], 2)*100)+'%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Predicted Sentiment', 'Polarity Score',
                                                                       'Positive', 'Negative',
                                                                       'Neutral']], 
                                                              labels=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
    
    return final_sentiment

In [882]:
for review, review_sentiment in sample_data:
    print('Review:')
    print( review)
    final_sentiment = analyze_sentiment_vader_lexicon(review,
                                                        threshold=0.1,
                                                        verbose=True)
    print('-'*60)

Review:
Was lucky enough to be an extra in this great film and loved every minute of the filming. Went to the premier in London and had a great chat to Phil, Peter, Martin, and Jon as did my wife.Fantastic after party too. Then a few weeks later had trip to the cinema with members of our bike club. What a brilliant film, it deserves to be up there with all the great biker films. Now we have the DVD Sue my wife can't get enough of it neither can the kids. Get a bit of stick from the club who seem to think I'm a film star now oh well one can only dream. I think they are just jealous. The only downside of the first part of the filming was the weather, rain, rain and more rain but hey we was in Wales. Hopefully there will be a follow up so keep me posted Jon. Danny Beck
     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0            positive           0.98    32.0%     5.0%   63.0%
----------------------------------

# Question 2

In [883]:
# 2. For each of the clusters you created in homework 7, compute the average, median, high, 
# and low sentiment scores for each cluster. Explain whether you think this reveals anything
# interesting about the clusters.

In [884]:
print('''one = pd.DataFrame(reviewData[reviewData['cluster'] == 0])
two = pd.DataFrame(reviewData[reviewData['cluster'] == 1])
three = pd.DataFrame(reviewData[reviewData['cluster'] == 2])
four = pd.DataFrame(reviewData[reviewData['cluster'] == 3])''')

one = pd.DataFrame(reviewData[reviewData['cluster'] == 0])
two = pd.DataFrame(reviewData[reviewData['cluster'] == 1])
three = pd.DataFrame(reviewData[reviewData['cluster'] == 2])
four = pd.DataFrame(reviewData[reviewData['cluster'] == 3])


In [885]:
# this is here for future work

#cluster one
#for review in one['review']:
#    print('Review:')
#    print( review)
#    final_sentiment = analyze_sentiment_vader_lexicon(review,
#                                                        threshold=0.1,
#                                                        verbose=True)
#    print('-'*60)

In [886]:
#cluster two
#for review in two['review']:
#    print('Review:')
#    print( review)
#    final_sentiment = analyze_sentiment_vader_lexicon(review,
#                                                        threshold=0.1,
#                                                        verbose=True)
#    print('-'*60)

In [887]:
#cluster three
#for review in three['review']:
#    print('Review:')
#    print( review)
#    final_sentiment = analyze_sentiment_vader_lexicon(review,
#                                                        threshold=0.1,
#                                                        verbose=True)
#    print('-'*60)

In [888]:
#cluster four
#for review in four['review']:
#    print('Review:')
#    print( review)
#    final_sentiment = analyze_sentiment_vader_lexicon(review,
#                                                        threshold=0.1,
#                                                        verbose=True)
#    print('-'*60)

In [889]:
#sentimentData1 = pd.DataFrame({'score': analyze_sentiment_vader_lexicon('Polarity Score')},
#                              index = one['cluster'])

In [890]:
#sentimentData2 = pd.DataFrame({'score': analyze_sentiment_vader_lexicon('Polarity Score')},
#                              index = two['cluster'])

In [891]:
#sentimentData3 = pd.DataFrame({'score': analyze_sentiment_vader_lexicon('Polarity Score')},
#                              index = three['cluster'])

In [892]:
#sentimentData4 = pd.DataFrame({'score': analyze_sentiment_vader_lexicon('Polarity Score')},
#                              index = four['cluster'])

In [893]:
# installing analyzer
!pip install vaderSentiment

tensorflow 1.12.0 has requirement keras-applications>=1.0.6, but you'll have keras-applications 1.0.4 which is incompatible.
tensorflow 1.12.0 has requirement keras-preprocessing>=1.0.5, but you'll have keras-preprocessing 1.0.2 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [894]:
analyser = SentimentIntensityAnalyzer()

sentences = one['review']

sent_scores_li = []

for sentence in sentences:

    sentiment_score = analyser.polarity_scores(sentence)
    print("-" * 10)
    print("THE REVIEW:", sentence)
    print("Negative sent. score:", sentiment_score["neg"])
    print("Neutral sent. score:", sentiment_score["neu"])
    print("Positive sent. score:", sentiment_score["pos"])
    print("Compund sent. score:", sentiment_score["compound"])

    compound_score = sentiment_score["compound"]

    sent_scores_li.append(compound_score)

scoreDataFrame_Cluster_1 = pd.DataFrame(sent_scores_li, columns=["VADER_compound_sentiment_score"])

----------
THE REVIEW: Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.
Negative sent. score: 0.144
Neutral sent. score: 0.756
Positive sent. score:

Neutral sent. score: 0.756
Positive sent. score: 0.1
Compund sent. score: -0.5349
----------
THE REVIEW: Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hou

----------
THE REVIEW: Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.
Negative sent. score: 0.144
Neutral sent. score: 0.756
Positive sent. score:

----------
THE REVIEW: Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.
Negative sent. score: 0.144
Neutral sent. score: 0.756
Positive sent. score:

Neutral sent. score: 0.756
Positive sent. score: 0.1
Compund sent. score: -0.5349
----------
THE REVIEW: Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hou

Positive sent. score: 0.1
Compund sent. score: -0.5349
----------
THE REVIEW: Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.
Negative sent. score:

Neutral sent. score: 0.756
Positive sent. score: 0.1
Compund sent. score: -0.5349
----------
THE REVIEW: Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hou

----------
THE REVIEW: Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.
Negative sent. score: 0.144
Neutral sent. score: 0.756
Positive sent. score:

Negative sent. score: 0.144
Neutral sent. score: 0.756
Positive sent. score: 0.1
Compund sent. score: -0.5349
----------
THE REVIEW: Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to kee

In [895]:
analyser = SentimentIntensityAnalyzer()

sentences = two['review']

sent_scores_li = []

for sentence in sentences:

    sentiment_score = analyser.polarity_scores(sentence)
    print("-" * 10)
    print("THE REVIEW:", sentence)
    print("Negative sent. score:", sentiment_score["neg"])
    print("Neutral sent. score:", sentiment_score["neu"])
    print("Positive sent. score:", sentiment_score["pos"])
    print("Compund sent. score:", sentiment_score["compound"])

    compound_score = sentiment_score["compound"]

    sent_scores_li.append(compound_score)

scoreDataFrame_Cluster_2 = pd.DataFrame(sent_scores_li, columns=["VADER_compound_sentiment_score"])

----------
THE REVIEW: This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat gratifying...<br /><br />Overall, this is second-rate action trash. There are countless better films to see, and i

Neutral sent. score: 0.819
Positive sent. score: 0.101
Compund sent. score: 0.6582
----------
THE REVIEW: This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat gratifying...<br /><br />Overal

----------
THE REVIEW: This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat gratifying...<br /><br />Overall, this is second-rate action trash. There are countless better films to see, and i

----------
THE REVIEW: This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat gratifying...<br /><br />Overall, this is second-rate action trash. There are countless better films to see, and i

----------
THE REVIEW: This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat gratifying...<br /><br />Overall, this is second-rate action trash. There are countless better films to see, and i

...

----------
THE REVIEW: This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat gratifying...<br /><br />Overall, this is second-rate action trash. There are countless better films to see, and i

Negative sent. score: 0.08
Neutral sent. score: 0.819
Positive sent. score: 0.101
Compund sent. score: 0.6582
----------
THE REVIEW: This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat grat

----------
THE REVIEW: This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat gratifying...<br /><br />Overall, this is second-rate action trash. There are countless better films to see, and i

Negative sent. score: 0.08
Neutral sent. score: 0.819
Positive sent. score: 0.101
Compund sent. score: 0.6582
----------
THE REVIEW: This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat grat

----------
THE REVIEW: This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat gratifying...<br /><br />Overall, this is second-rate action trash. There are countless better films to see, and i

In [896]:
analyser = SentimentIntensityAnalyzer()

sentences = three['review']

sent_scores_li = []

for sentence in sentences:

    sentiment_score = analyser.polarity_scores(sentence)
    print("-" * 10)
    print("THE REVIEW:", sentence)
    print("Negative sent. score:", sentiment_score["neg"])
    print("Neutral sent. score:", sentiment_score["neu"])
    print("Positive sent. score:", sentiment_score["pos"])
    print("Compund sent. score:", sentiment_score["compound"])

    compound_score = sentiment_score["compound"]

    sent_scores_li.append(compound_score)

scoreDataFrame_Cluster_3 = pd.DataFrame(sent_scores_li, columns=["VADER_compound_sentiment_score"])

----------
THE REVIEW: First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangsters.<br /><br />The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.<br /><br />Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.<br /><br />The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody get what they deserve.<br /><br />The only two black peopl

----------
THE REVIEW: First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangsters.<br /><br />The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.<br /><br />Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.<br /><br />The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody get what they deserve.<br /><br />The only two black peopl

Neutral sent. score: 0.797
Positive sent. score: 0.073
Compund sent. score: -0.897
----------
THE REVIEW: First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangsters.<br /><br />The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.<br /><br />Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.<br /><br />The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the ho

----------
THE REVIEW: First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangsters.<br /><br />The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.<br /><br />Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.<br /><br />The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody get what they deserve.<br /><br />The only two black peopl

Positive sent. score: 0.073
Compund sent. score: -0.897
----------
THE REVIEW: First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangsters.<br /><br />The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.<br /><br />Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.<br /><br />The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody ge

Positive sent. score: 0.073
Compund sent. score: -0.897
----------
THE REVIEW: First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangsters.<br /><br />The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.<br /><br />Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.<br /><br />The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody ge

----------
THE REVIEW: First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangsters.<br /><br />The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.<br /><br />Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.<br /><br />The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody get what they deserve.<br /><br />The only two black peopl

THE REVIEW: First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangsters.<br /><br />The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.<br /><br />Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.<br /><br />The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody get what they deserve.<br /><br />The only two black people that can 

In [897]:
analyser = SentimentIntensityAnalyzer()

sentences = four['review']

sent_scores_li = []

for sentence in sentences:

    sentiment_score = analyser.polarity_scores(sentence)
    print("-" * 10)
    print("THE REVIEW:", sentence)
    print("Negative sent. score:", sentiment_score["neg"])
    print("Neutral sent. score:", sentiment_score["neu"])
    print("Positive sent. score:", sentiment_score["pos"])
    print("Compund sent. score:", sentiment_score["compound"])

    compound_score = sentiment_score["compound"]

    sent_scores_li.append(compound_score)

scoreDataFrame_Cluster_4 = pd.DataFrame(sent_scores_li, columns=["VADER_compound_sentiment_score"])

----------
THE REVIEW: Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social platforms were changing in music and film, the emergence of the Rapper turned movie star was in full swing, the acting took a back seat to each man's overpowering regional accent and transparent acting. This was one of the many ice-t movies i saw as a kid and loved, only to watch them later and cringe. Bill Paxton and William Sadler are firemen with basic lives until a burning building tenant about to go up in flames hands over a map with gold implications. I hand it to Walter for quickly and neatly setting up the main characters and location. But i fault everyone involved for turning out Lame-o performances. Ice-t and cube must have been red hot at this time, and while I've enjoyed both their careers as rappers, in my opinion they fell flat in this movie. It's about ninety

----------
THE REVIEW: Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social platforms were changing in music and film, the emergence of the Rapper turned movie star was in full swing, the acting took a back seat to each man's overpowering regional accent and transparent acting. This was one of the many ice-t movies i saw as a kid and loved, only to watch them later and cringe. Bill Paxton and William Sadler are firemen with basic lives until a burning building tenant about to go up in flames hands over a map with gold implications. I hand it to Walter for quickly and neatly setting up the main characters and location. But i fault everyone involved for turning out Lame-o performances. Ice-t and cube must have been red hot at this time, and while I've enjoyed both their careers as rappers, in my opinion they fell flat in this movie. It's about ninety

----------
THE REVIEW: Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social platforms were changing in music and film, the emergence of the Rapper turned movie star was in full swing, the acting took a back seat to each man's overpowering regional accent and transparent acting. This was one of the many ice-t movies i saw as a kid and loved, only to watch them later and cringe. Bill Paxton and William Sadler are firemen with basic lives until a burning building tenant about to go up in flames hands over a map with gold implications. I hand it to Walter for quickly and neatly setting up the main characters and location. But i fault everyone involved for turning out Lame-o performances. Ice-t and cube must have been red hot at this time, and while I've enjoyed both their careers as rappers, in my opinion they fell flat in this movie. It's about ninety

THE REVIEW: Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social platforms were changing in music and film, the emergence of the Rapper turned movie star was in full swing, the acting took a back seat to each man's overpowering regional accent and transparent acting. This was one of the many ice-t movies i saw as a kid and loved, only to watch them later and cringe. Bill Paxton and William Sadler are firemen with basic lives until a burning building tenant about to go up in flames hands over a map with gold implications. I hand it to Walter for quickly and neatly setting up the main characters and location. But i fault everyone involved for turning out Lame-o performances. Ice-t and cube must have been red hot at this time, and while I've enjoyed both their careers as rappers, in my opinion they fell flat in this movie. It's about ninety minutes of

Negative sent. score: 0.067
Neutral sent. score: 0.823
Positive sent. score: 0.11
Compund sent. score: 0.9306
----------
THE REVIEW: Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social platforms were changing in music and film, the emergence of the Rapper turned movie star was in full swing, the acting took a back seat to each man's overpowering regional accent and transparent acting. This was one of the many ice-t movies i saw as a kid and loved, only to watch them later and cringe. Bill Paxton and William Sadler are firemen with basic lives until a burning building tenant about to go up in flames hands over a map with gold implications. I hand it to Walter for quickly and neatly setting up the main characters and location. But i fault everyone involved for turning out Lame-o performances. Ice-t and cube must have been red hot at this time, and w

----------
THE REVIEW: Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social platforms were changing in music and film, the emergence of the Rapper turned movie star was in full swing, the acting took a back seat to each man's overpowering regional accent and transparent acting. This was one of the many ice-t movies i saw as a kid and loved, only to watch them later and cringe. Bill Paxton and William Sadler are firemen with basic lives until a burning building tenant about to go up in flames hands over a map with gold implications. I hand it to Walter for quickly and neatly setting up the main characters and location. But i fault everyone involved for turning out Lame-o performances. Ice-t and cube must have been red hot at this time, and while I've enjoyed both their careers as rappers, in my opinion they fell flat in this movie. It's about ninety

----------
THE REVIEW: Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social platforms were changing in music and film, the emergence of the Rapper turned movie star was in full swing, the acting took a back seat to each man's overpowering regional accent and transparent acting. This was one of the many ice-t movies i saw as a kid and loved, only to watch them later and cringe. Bill Paxton and William Sadler are firemen with basic lives until a burning building tenant about to go up in flames hands over a map with gold implications. I hand it to Walter for quickly and neatly setting up the main characters and location. But i fault everyone involved for turning out Lame-o performances. Ice-t and cube must have been red hot at this time, and while I've enjoyed both their careers as rappers, in my opinion they fell flat in this movie. It's about ninety

Negative sent. score: 0.067
Neutral sent. score: 0.823
Positive sent. score: 0.11
Compund sent. score: 0.9306
----------
THE REVIEW: Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social platforms were changing in music and film, the emergence of the Rapper turned movie star was in full swing, the acting took a back seat to each man's overpowering regional accent and transparent acting. This was one of the many ice-t movies i saw as a kid and loved, only to watch them later and cringe. Bill Paxton and William Sadler are firemen with basic lives until a burning building tenant about to go up in flames hands over a map with gold implications. I hand it to Walter for quickly and neatly setting up the main characters and location. But i fault everyone involved for turning out Lame-o performances. Ice-t and cube must have been red hot at this time, and w

In [919]:
print(
"Mean Scores:",
scoreDataFrame_Cluster_1.mean(),
scoreDataFrame_Cluster_2.mean(),
scoreDataFrame_Cluster_3.mean(),
scoreDataFrame_Cluster_4.mean())

Mean Scores: VADER_compound_sentiment_score   -0.5349
dtype: float64 VADER_compound_sentiment_score    0.6582
dtype: float64 VADER_compound_sentiment_score   -0.897
dtype: float64 VADER_compound_sentiment_score    0.9306
dtype: float64


In [933]:
# min
scoreDataFrame_Cluster_4.min()
scoreDataFrame_Cluster_1.min(),
scoreDataFrame_Cluster_2.min(),
scoreDataFrame_Cluster_3.min()

VADER_compound_sentiment_score   -0.897
dtype: float64

In [934]:
# max
scoreDataFrame_Cluster_1.max(),
scoreDataFrame_Cluster_2.max(),
scoreDataFrame_Cluster_3.max(),
scoreDataFrame_Cluster_4.max()

VADER_compound_sentiment_score    0.9306
dtype: float64

In [935]:
# median
scoreDataFrame_Cluster_3.median(),
scoreDataFrame_Cluster_4.median(),
scoreDataFrame_Cluster_1.median(),
scoreDataFrame_Cluster_2.median()

VADER_compound_sentiment_score    0.6582
dtype: float64

In [937]:
# Analysis
print('''
Cluster 1 is medium negative
Cluster 2 is medium positive
Cluster 3 is high negative
Cluster 4 is high positive
It seems that the sentiment analzyer did it properly and even with a well distribution of scores. Each cluster
is very seperate from the others (such as a BGSS - Between Group Sum of Squares). Cluster 1 and Cluster 3 are more
similar to one another because they are in the negative facet, while Cluster 2 and Cluster 4 are similar to each
other in that they are both in the positive facet. One of the interesting findings about these clusters is that
they are correct and incorrect in their sentiment summary. For example, "No magic here, it was all I could do to 
keep from turning it off an hour in." was analzyed correctly in a negative review for Cluster 1. The next cluster,
Cluster 2 was also pretty close to predicting correctly: "the cinematography was almost refreshing, which comes 
close to making up for the horrible film itself - but not quite. 4/10". In this case, it had the actual score in the
text review while giving confusing 'refreshing' and 'horrible' terms in the very same sentence, but the model picked
it up correctly. Cluster 3 had the word "hate" and that definitely correlated correctly with a highly negative
review.
Lastly, unlike all the correct clusters, Cluster 4 was way off. It labeled as highly positive, with a review like
"I'm well aware it tastes like crap. 3 stars, meh.". One reason for this miss classification is possibly that it had
terms like 'gold' and 'liked' that threw off the sentiment analysis scoring.
''')


Cluster 1 is medium negative
Cluster 2 is medium positive
Cluster 3 is high negative
Cluster 4 is high positive
It seems that the sentiment analzyer did it properly and even with a well distribution of scores. Each cluster
is very seperate from the others (such as a BGSS - Between Group Sum of Squares). Cluster 1 and Cluster 3 are more
similar to one another because they are in the negative facet, while Cluster 2 and Cluster 4 are similar to each
other in that they are both in the positive facet. One of the interesting findings about these clusters is that
they are correct and incorrect in their sentiment summary. For example, "No magic here, it was all I could do to 
keep from turning it off an hour in." was analzyed correctly in a negative review for Cluster 1. The next cluster,
Cluster 2 was also pretty close to predicting correctly: "the cinematography was almost refreshing, which comes 
close to making up for the horrible film itself - but not quite. 4/10". In this case, it had t